In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


TODO: adjust the following paths

In [ ]:
#data_path = ".../Data/ARTS/laptop_test.json"
#output_path = ".../Data/Final/ARTS-LAP/"
#aspect_path = ".../preprocessing/aspect_positions/"

# Data Import

In [2]:
import pandas as pd

In [3]:
df = pd.read_json(data_path)
df = df.transpose()
df.reset_index(inplace=True,drop=True)
df.rename(columns={"sentence":"text"},inplace=True)

In [4]:
df

,text,term,polarity,id,from,to
0,3D rendering slows it down considerably.,3D rendering,negative,0:14_0,0,12
1,3D rendering not slows it down considerably.,3D rendering,positive,0:14_0,0,12
2,"3D rendering slows it down considerably, but k...",3D rendering,negative,0:14_0,0,12
3,"I'm using this computer for word processing, w...",word processing,neutral,0:1_0,28,43
4,"I'm using this computer for word processing, w...",word processing,neutral,0:1_0,28,43
...,...,...,...,...,...,...
1872,"However, there are MAJOR not issues with the t...",touchpad,positive,998:1_0,45,53
1873,"However, there are MAJOR issues with the touch...",touchpad,negative,998:1_0,41,49
1874,The smaller size was a bonus because of space ...,size,positive,999:1_0,12,16
1875,The bigger size was not a bonus because of spa...,size,negative,999:1_0,11,15


# Drop duplicates

In [5]:
cols_wo_id = df.columns
cols_wo_id = cols_wo_id.drop("id")
df[df.duplicated(cols_wo_id, keep=False)].sort_values("text")

,text,term,polarity,id,from,to


In [6]:
df.drop_duplicates(cols_wo_id, ignore_index=True,inplace=True)

In [7]:
text_counts = df.text.value_counts()
text_counts[:10]

also...- excellent operating system- size and weight for optimal mobility- excellent durability of the battery- the functions provided by the trackpad is unmatched by any other brand-                                                                                                                                                 6
Has all the other features I wanted including a VGA port, HDMI, ethernet and 3 USB ports.                                                                                                                                                                                                                                               5
In fact I still use many Legacy programs (Appleworks, FileMaker Pro, Quicken, Photoshop etc)!                                                                                                                                                                                                                                           5
The Like N

# Descriptive Analysis


## Repeating Sentences

In [8]:
len(df.text.value_counts())

1576

In [9]:
df_sent_freq = df.text.value_counts()
df_sent_freq

also...- excellent operating system- size and weight for optimal mobility- excellent durability of the battery- the functions provided by the trackpad is unmatched by any other brand-                                                                                                                                                 6
Has all the other features I wanted including a VGA port, HDMI, ethernet and 3 USB ports.                                                                                                                                                                                                                                               5
In fact I still use many Legacy programs (Appleworks, FileMaker Pro, Quicken, Photoshop etc)!                                                                                                                                                                                                                                           5
The Like N

## Sentiment Frequency

In [10]:
df.polarity.value_counts()

positive    883
negative    587
neutral     407
Name: polarity, dtype: int64

In [11]:
sum(df.polarity.value_counts())

1877

## Sentiments of repeating sentences

In [12]:
df_s = pd.DataFrame(df_sent_freq)
df_s.reset_index(inplace=True)
df_s = df_s.rename(columns={"index":"sentence", "text": "freq"})
df_s

,sentence,freq
0,also...- excellent operating system- size and ...,6
1,Has all the other features I wanted including ...,5
2,In fact I still use many Legacy programs (Appl...,5
3,The Like New condition of the iMac MC309LL/A o...,5
4,A veryimportant feature is Firewire 800 which ...,5
...,...,...
1571,From the speed to the multi touch gestures thi...,1
1572,the volume is really low to low for a laptopwa...,1
1573,"WiFi capability, disk drive and multiple USB p...",1
1574,"The OS is easy, and offers all kinds of surpri...",1


In [13]:
df_s.freq.value_counts()

1    1360
2     159
3      37
4      13
5       6
6       1
Name: freq, dtype: int64

In [14]:
sum(df_s.freq.value_counts())

1576

## Sentences with at least two aspects

In [15]:
df_min2asp = df_s[df_s.freq > 1]
df_min2asp

,sentence,freq
0,also...- excellent operating system- size and ...,6
1,Has all the other features I wanted including ...,5
2,In fact I still use many Legacy programs (Appl...,5
3,The Like New condition of the iMac MC309LL/A o...,5
4,A veryimportant feature is Firewire 800 which ...,5
...,...,...
211,Plus two finger clicking as a replacement for ...,2
212,They don't just look good; they deliver excell...,2
213,Everything is so easy and intuitive to setup o...,2
214,"However, it did not have any scratches, ZERO b...",2


Subset with at different sentiments.

In [16]:
diff_sents = 0
for sent in df_min2asp.sentence:
    if len(df[df.text == sent].polarity.value_counts()) > 1:
        diff_sents += 1
diff_sents

74

# Check for wrong positions

In [17]:
mistakes = []
for ii in range(len(df)):
    actual_term = df.term[ii]
    pos_term = df.text[ii][df.loc[ii,"from"]:df.to[ii]]
    if actual_term != pos_term:
        mistakes += [ii]
        print(actual_term, pos_term)
mistakes

startup disk Startup disk
"tools" menu " tools " me
"Home/End" type keys "Home/End"


[528, 608, 1725]

In [18]:
df.loc[528,"term"] = "Startup disk"

In [19]:
df.loc[608,"to"] += 2
df.loc[608,"term"] = '" tools " menu'

In [20]:
df.loc[1725,"to"] += 10

# Check for wrong aspect terms

In [21]:
import nltk

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
for ii in range(len(df)):
    tokens = nltk.word_tokenize(df.text[ii])
    actual_term = df.term[ii]
    for asp_part in nltk.word_tokenize(actual_term):
        if asp_part not in tokens and asp_part+"-" not in tokens:
            print(ii,":",tokens)
            print(actual_term, asp_part)

515 : ['Here', 'are', 'the', 'things', 'that', 'made', 'me', 'confident', 'with', 'my', 'purchase', ':', 'Build', 'Quality-Seriously', ',', 'you', 'can', 'beat', 'a', 'unibody', 'construction', '.']
Build Quality Quality
534 : ['Screen-although', 'some', 'people', 'might', 'complain', 'about', 'low', 'res', 'which', 'I', 'think', 'is', 'not', 'ridiculous']
Screen Screen
856 : ['fast', ',', 'great', 'screen', ',', 'beautiful', 'apps', 'for', 'a', 'laptop', ';', 'priced', 'at', '1100', 'on', 'the', 'apple', 'website', ';', 'amazon', 'had', 'it', 'for', '1098+', 'tax', '-', 'plus', 'i', 'had', 'a', '10', '%', 'off', 'coupon', 'from', 'amazon-cost', 'me', '998', 'plus', 'tax-', '1070-', 'OTD', '!']
cost cost
857 : ['fast', ',', 'great', 'screen', ',', 'beautiful', 'apps', 'for', 'a', 'laptop', ';', 'priced', 'at', '1100', 'on', 'the', 'apple', 'website', ';', 'amazon', 'had', 'it', 'for', '1098+', 'tax', '-', 'plus', 'i', 'had', 'a', '10', '%', 'off', 'coupon', 'from', 'amazon-cost', 'me',

# Convert ID to int

In [ ]:
import numpy as np

In [ ]:
for line in df.index:
    temp = df.id[line].split("_")[0]
    temp = temp.replace(":", ".")
    df.id[line] = np.float(temp)

# Save as CapsNetBERTxml

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def xml_maker(df):

    root = ET.Element('sentences')

    for line in df.index:
        name = "sentence"
        entry = ET.SubElement(root, name)
        entry.set("id", str(df["id"][line]))

        text_child = ET.SubElement(entry, "text")
        text_child.text = str(df["text"][line])

        asp_child = ET.SubElement(entry, "aspectTerms")
        asp_subchild = ET.SubElement(asp_child, "aspectTerm")
        asp_subchild.set("from",str(df["from"][line]))
        asp_subchild.set("polarity",str(df["polarity"][line]))
        asp_subchild.set("term",str(df["term"][line]))
        asp_subchild.set("to",str(df["to"][line]))

    return ET.tostring(root)

In [ ]:
xml_data = xml_maker(df)

with open(output_path+"test.xml","w") as f:
    f.write(xml_data.decode('utf-8'))

# Create MGATNxml.seg

In [ ]:
def pol_to_no(sentiment):
  
    if sentiment == "positive":
        pol = 1
    elif sentiment == "negative":
        pol = -1
    elif sentiment == "neutral":
        pol = 0

    return str(pol)

In [ ]:
def xml_seg_maker(df):

    data_lines = []

    for ii in range(len(df)):
        text = df.text[ii].replace(df.term[ii], "$T$")
        pol = pol_to_no(df.polarity[ii])

        data_lines += [text, df.term[ii], pol, str(df.id[ii])]

    return data_lines

In [ ]:
xml_seg_data = xml_seg_maker(df)

with open(output_path+"test.xml.seg","w") as f:
    f.write('\n'.join(xml_seg_data))

# Create BERT+txt

In [ ]:
def sent_conv(sentiment):

    if sentiment == "positive":
        return "POS"
    elif sentiment == "negative":
        return "NEG"
    elif sentiment == "neutral":
        return "NEU"

In [ ]:
from more_itertools import locate

In [ ]:
def txt_maker(df):    
    
    sent_freq = df.text.value_counts()
    data_lines = []

    for text,freq in sent_freq.items():
        
        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # find corresponding lines in df
        text_lines = list(locate(list(df.text), lambda a: a == text))
        assert len(text_lines) == freq

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        for line in text_lines:
            aspect = df.loc[line,"term"]
            if aspect != None:
                asp_sent_dict[aspect] = sent_conv(df.loc[line,"polarity"])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + "=T-" + asp_sent_dict[tok] + " "
            else:
                label += tok + "=O "
        label = label[:-1]

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    old_label = " ".join([tokens[no+xx]+"=O" for xx in range(ii) if no+xx < len(tokens)])
                    new_label = " ".join([tokens[no+xx]+"=T-"+new_pol for xx in range(ii) if no+xx < len(tokens)])
                    label = label.replace(old_label, new_label)

        # get ids
        id = []
        for line in text_lines:
            id += [df.loc[line, "id"]]

        assert(len(set(id)) == 1)

        data_lines += [text+"####"+label+"####"+str(id[0])]

    return data_lines

In [ ]:
txt_data = txt_maker(df)

with open(output_path+"test.txt","w") as f:
    f.write('\n'.join(txt_data))

# Create RGATjson

In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 13.4MB/s 


In [ ]:
import stanza
from more_itertools import locate
import json

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,depparse')

2021-06-07 09:37:07 INFO: Downloading default packages for language: en (English)...
2021-06-07 09:38:30 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-06-07 09:38:30 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-06-07 09:38:30 INFO: Use device: cpu
2021-06-07 09:38:30 INFO: Loading: tokenize
2021-06-07 09:38:30 INFO: Loading: pos
2021-06-07 09:38:30 INFO: Loading: lemma
2021-06-07 09:38:30 INFO: Loading: depparse
2021-06-07 09:38:31 INFO: Done loading processors!


## in case of creating for the first time

In [ ]:
'''
def json_make_pos(df):

    manual_pos = {}
    sent_freq = df.text.value_counts()

    for text,freq in sent_freq.items():
        
        new_dict = {}
        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        
        new_dict["aspects"] = []
        text_lines = list(locate(list(df.text), lambda a: a == text))

        for line in text_lines:
            asp_dict = {}   
            term = df.loc[line,"term"]

            if term != None:
                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == tok)) for tok in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:
                    print(line, ": ", text)
                    print("original term: ",term)
                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    print("all start chars: ",all_char_from)
                    corr_char_from = int(df.loc[line, "from"])
                    print("correct start char: ",corr_char_from)
                    print("text beginning at correct start char: ", text[corr_char_from:])
                    print("tokens: ",tokens)
                    print("original asp tokens: ", asp_toks)
                    print("original asp indices: ",asp_ind)
                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:

                    print("Tokenization Error in line ",line,"!")
                    for pos, tok in enumerate(tokens):
                        print(pos, tok)
                    print("aspect term: ", term)
                    print("original asp tokens: ", asp_toks)

                    from_index = int(input("start position?"))
                    to_index = int(input("end position?"))

                    # add manually stated positions to dict for reproducibility
                    manual_pos[text] = {}
                    manual_pos[text][term] = {}
                    manual_pos[text][term]["from"] = from_index
                    manual_pos[text][term]["to"] = to_index
                    print(manual_pos)

                    print("Final aspect tokens: ", tokens[from_index:to_index+1])

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

    with open(aspect_path+"arts_lap.json","w") as f:
        json.dump(manual_pos, f)

json_make_pos(df)
'''

1537 :  A veryimportant feature is Firewire 800 which in my experience works better then USB3 (in PC enabled with USB3)I was not originally sold on the MAC OS I felt it was inferior in many ways To Windows 7.
original term:  USB3
all start chars:  [81, 106]
correct start char:  81
text beginning at correct start char:  USB3 (in PC enabled with USB3)I was not originally sold on the MAC OS I felt it was inferior in many ways To Windows 7.
tokens:  ['A', 'veryimportant', 'feature', 'is', 'Firewire', '800', 'which', 'in', 'my', 'experience', 'works', 'better', 'then', 'USB3', '(', 'in', 'PC', 'enabled', 'with', 'USB3', ')', 'I', 'was', 'not', 'originally', 'sold', 'on', 'the', 'MAC', 'OS', 'I', 'felt', 'it', 'was', 'inferior', 'in', 'many', 'ways', 'To', 'Windows', '7', '.']
original asp tokens:  ['USB3']
original asp indices:  [[13, 19]]
1547 :  A veryimportant feature is Firewire 800 which in my experience works better then USB3 (in PC enabled with USB3)I was not originally sold on the M

## in case of reproducing the dataset

In [ ]:
def json_maker(df, manual_pos):
   
    new_data = []
    sent_freq = df.text.value_counts()

    for text,freq in sent_freq.items():
        
        new_dict = {}

        tokens = [token.text for sentence in nlp(text).sentences for token in sentence.tokens]
        new_dict["token"] = tokens

        new_dict["pos"] = [word.xpos for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["head"] = [str(word.head) for sentence in nlp(text).sentences for word in sentence.words]
        new_dict["deprel"] = [word.deprel for sentence in nlp(text).sentences for word in sentence.words]
        
        new_dict["aspects"] = []
        text_lines = list(locate(list(df.text), lambda a: a == text))

        ids = []
        for line in text_lines:
            asp_dict = {}   
            term = df.loc[line,"term"]
            ids += [df.loc[line,"id"]]

            if term != None:
                asp_dict["term"] = term
                asp_dict["polarity"] = df.loc[line,"polarity"]

                # construct aspect position on token level
                asp_toks = [token.text for sentence in nlp(term).sentences for token in sentence.tokens]
                asp_ind = [list(locate(tokens, lambda a: a == tok)) for tok in asp_toks]

                # for aspects appearing only once in text, take the correct position
                # otherwise set to None
                if len(asp_ind[0]) == 1:
                    from_index = asp_ind[0][0]
                else:
                    from_index = None
                if len(asp_ind[-1]) == 1:
                    to_index = asp_ind[-1][0] 
                else: 
                    to_index = None

                # if both start and end pos are unknown, 
                # e.g. for single-word aspects, 
                # take character positions for help
                if from_index == None and to_index == None and len(asp_ind[0]) != 0:

                    all_char_from = [i for i in range(len(text)) if text.startswith(asp_toks[0], i)]
                    corr_char_from = int(df.loc[line, "from"])

                    if corr_char_from == max(all_char_from):
                        from_index = asp_ind[0][-1]
                    elif corr_char_from == min(all_char_from):
                        from_index = asp_ind[0][0]

                # in case of missing start/end positions,
                # try to find "to"/"from" using aspect token number as distance
                if from_index == None and to_index != None:
                    from_index = to_index - len(asp_toks) +1
                if to_index == None and from_index != None:
                    to_index = from_index + len(asp_toks) -1

                # correct tokenization errors in aspect term tokenization
                if from_index == None or to_index == None or asp_toks != tokens[from_index:to_index+1]:
                    if text in manual_pos.keys() and term in manual_pos[text].keys():
                        from_index = manual_pos[text][term]["from"]
                        to_index = manual_pos[text][term]["to"]

                asp_dict["from"] = from_index
                asp_dict["to"] = to_index + 1

            new_dict["aspects"] += [asp_dict]
        
        assert len(set(ids)) == 1
        new_dict["id"] = ids[0]

        new_data += [new_dict]

    return new_data

In [ ]:
pos_obj = open(aspect_path+"arts_lap.json")
loaded_pos = json.load(pos_obj)

json_data = json_maker(df, loaded_pos)

with open(output_path+"test.json","w") as f:
    json.dump(json_data, f)

# Create LCF-ATEPCdat

In [ ]:
def pol_to_no_shifted(sentiment):
  
    if sentiment == "positive":
        pol = 2
    elif sentiment == "negative":
        pol = 0
    elif sentiment == "neutral":
        pol = 1

    return str(pol)

In [ ]:
def dat_maker(df):
    
    data_lines = []
    sent_freq = df.text.value_counts()

    for text,freq in sent_freq.items():

        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        text_lines = list(locate(list(df.text), lambda a: a == text))

        # create aspect-polarity dict
        asp_sent_dict = {}
        ids = []
        max_asp_len = 0
        for line in text_lines:
            aspect = df.loc[line,"term"]
            ids += [df.loc[line,"id"]]
            if aspect != None:
                asp_sent_dict[aspect] = pol_to_no_shifted(df.loc[line,"polarity"])
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        assert len(set(ids)) == 1

        label = ""
        # check for one-word-aspects
        for tok in tokens:
            if tok in asp_sent_dict.keys():
                label += tok + " B-ASP -1\n"
            else:
                label += tok + " O -1\n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    label = label.replace(tokens[no]+" O -1",tokens[no]+" B-ASP -1")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx]+" O -1",tokens[no+xx]+" I-ASP -1")

        # create duplicates of review in case of more than one aspect
        for key, val in asp_sent_dict.items():
            if key in tokens:
                new_label = label.replace(key+" B-ASP -1", key+" B-ASP "+val)
                data_lines += [new_label]
                data_lines += [str(ids[0])+"\n\n"]
            else:
                for ii in range(2,max_asp_len+1):
                    for no,tok in enumerate(tokens):
                        new_tok = " ".join(tokens[no:no+ii])
                        if new_tok == key:
                            new_label = label.replace(tokens[no]+" B-ASP -1",tokens[no]+" B-ASP "+val)
                            for xx in range(1,ii):
                                new_label = new_label.replace(tokens[no+xx]+" I-ASP -1",tokens[no+xx]+" I-ASP "+val)

                            data_lines += [new_label]
                            data_lines += [str(ids[0])+"\n\n"]

    return data_lines

In [ ]:
dat_data = dat_maker(df)

with open(output_path+"test.dat","w") as f:
    f.write(''.join(dat_data))

# Create GRACEtxt

In [ ]:
import nltk

In [ ]:
from more_itertools import locate

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Create chunks and pos tags. Source: https://towardsdatascience.com/chunking-in-nlp-decoded-b4a71b2b4e24


In [ ]:
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI
from nltk.chunk.util import tree2conlltags, conlltags2tree
from nltk.corpus import conll2000

In [ ]:
def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
    
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

class NGramTagChunker(ChunkParserI):

    def __init__(self,train_sentences,tagger_classes=[UnigramTagger,BigramTagger]):
        train_sent_tags=conll_tag_chunks(train_sentences)
        self.chunk_tagger=combined_tagger(train_sent_tags,tagger_classes)
    
    def parse(self,tagged_sentence):
        if not tagged_sentence:
            return None
        pos_tags=[tag for word, tag in tagged_sentence]
        chunk_pos_tags=self.chunk_tagger.tag(pos_tags)
        chunk_tags=[chunk_tag for (pos_tag,chunk_tag) in chunk_pos_tags]
        wpc_tags=[(word,pos_tag,chunk_tag) for ((word,pos_tag),chunk_tag) in zip(tagged_sentence,chunk_tags)]
        return conlltags2tree(wpc_tags)

In [ ]:
data = conll2000.chunked_sents()
ntc = NGramTagChunker(data)

Convert tags into BIOES scheme. Source: https://gist.github.com/allanj/5ad206f7f4645c0269b68fb2065712f4

In [ ]:
def iob_iobes(tags):
    """
    IOB2 (BIO) -> IOBES
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag == 'O':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'B':
            if i + 1 != len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('B-', 'S-'))
        elif tag.split('-')[0] == 'I':
            if i + 1 < len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('I-', 'E-'))
        else:
            raise Exception('Invalid IOB format!')
    return new_tags

In [ ]:
def grace_txt_maker(df):
    
    data_lines = []
    sent_freq = df.text.value_counts()

    for text,freq in sent_freq.items():

        tokens = nltk.word_tokenize(text)

        # correct tokens
        for no,tok in enumerate(tokens):
            if tok[-1:] == "-" and len(tok)>2:
                tokens[no] = tok[:-1]
            if tok[:1] == "'" and len(tok)>3:
                tokens[no] = tok[1:]
            if tok in ["'","(",")"]:
                tokens.remove(tok)

        # create pos tags
        pos_tags = nltk.pos_tag(tokens)

        # create chunk/phrase tags
        full_tags = tree2conlltags(ntc.parse(pos_tags))
        chunks_list = [full_tags[ii][2] for ii in range(len(full_tags))]
        new_chunks = iob_iobes(chunks_list)

        text_lines = list(locate(list(df.text), lambda a: a == text))

        # create aspect-polarity dict
        asp_sent_dict = {}
        max_asp_len = 0
        ids = []
        for line in text_lines:
            ids += [df.loc[line,"id"]]
            aspect = df.loc[line,"term"]
            if aspect != None:
                asp_sent_dict[aspect] = df.loc[line,"polarity"].upper()
                if len(aspect.split()) > max_asp_len:
                    max_asp_len = len(aspect.split())

        label = ""
        # check for one-word-aspects
        for pos,tok in enumerate(tokens):
            label += tok + " " + pos_tags[pos][1] + " " + new_chunks[pos]
            if tok in asp_sent_dict.keys():
                label +=  " B_AP " + asp_sent_dict[tok] + " B_AP+" + asp_sent_dict[tok] + "\n"
            else:
                label += " O O O \n"

        # check for multi-word-aspects
        for ii in range(2,max_asp_len+1):
            for no,tok in enumerate(tokens):
                new_tok = " ".join(tokens[no:no+ii])
                if new_tok not in tokens and new_tok in asp_sent_dict.keys():
                    new_pol = asp_sent_dict[new_tok]
                    label = label.replace(tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " O O O \n",
                                          tokens[no]+ " " + pos_tags[no][1] + " " + new_chunks[no] + " B_AP " + \
                                          new_pol + " B_AP+" + new_pol + "\n")
                    for xx in range(1,ii):
                        label = label.replace(tokens[no+xx] + " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " O O O \n",
                                          tokens[no+xx]+ " " + pos_tags[no+xx][1] + " " + new_chunks[no+xx] + " I_AP " + \
                                          new_pol + " I_AP+" + new_pol + "\n")

        assert len(set(ids)) == 1
        label += str(ids[0]) + "\n"
        data_lines += [label]

    return data_lines

In [ ]:
grace_txt_data = grace_txt_maker(df)

with open(output_path+"grace_test.txt","w") as f:
    f.write('\n'.join(grace_txt_data))